<a href="https://colab.research.google.com/github/Ataraxia1001/Login_project/blob/main/Data_analysis_visualization/busstop_total_population_monthly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⭐️ 버스 정류소 월별 승하차 인원 합계 및 정류소 위치 정보
- 목표: 월평균 유동인구가 많은 버스정류장의 위치 데이터
  - 1단계 : 데이터 전처리
    - 필요없는 컬럼 삭제
    - 병합을 위해 컬럼 이름 통일 및 데이터 타입 일치시키기
    - 해당 자료는 시간대별 승하차 인구를 모두 포함함
      - 시간대별 유동인 수가 필요한 것이 아니라 월평균 총유동인구 수가 필요하므로 정류장별 월평균유동인구 값을 구하기(승차, 하차 구분)
  - 2단계 : 정류소별 월, 시간대별 승하차 인원 수를 포함하는 데이터프레임과 버스 정류장의 위치 좌표를 가진 데이터프레임 병합

## ✅ 데이터 전처리

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install haversine
!pip install tqdm

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
from haversine import haversine

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/project1_login/data/bus_station_boarding/2022년_버스노선별_정류장별_시간대별_승하차_인원_정보(01월).csv', encoding='cp949')
df2 = pd.read_csv('/content/drive/MyDrive/project1_login/data/bus_station_boarding/2022년_버스노선별_정류장별_시간대별_승하차_인원_정보(02월).csv', encoding='cp949')
df3 = pd.read_csv('/content/drive/MyDrive/project1_login/data/bus_station_boarding/2022년_버스노선별_정류장별_시간대별_승하차_인원_정보(03월).csv', encoding='cp949')
df4 = pd.read_csv('/content/drive/MyDrive/project1_login/data/bus_station_boarding/2022년_버스노선별_정류장별_시간대별_승하차_인원_정보(04월).csv', encoding='cp949')
df5 = pd.read_csv('/content/drive/MyDrive/project1_login/data/bus_station_boarding/2022년_버스노선별_정류장별_시간대별_승하차_인원_정보(05월).csv', encoding='cp949')
df6 = pd.read_csv('/content/drive/MyDrive/project1_login/data/bus_station_boarding/2022년_버스노선별_정류장별_시간대별_승하차_인원_정보(06월).csv', encoding='cp949')
df7 = pd.read_csv('/content/drive/MyDrive/project1_login/data/bus_station_boarding/2022년_버스노선별_정류장별_시간대별_승하차_인원_정보(07월).csv', encoding='cp949')
df8 = pd.read_csv('/content/drive/MyDrive/project1_login/data/bus_station_boarding/2022년_버스노선별_정류장별_시간대별_승하차_인원_정보(08월).csv', encoding='cp949')
df9 = pd.read_csv('/content/drive/MyDrive/project1_login/data/bus_station_boarding/2022년_버스노선별_정류장별_시간대별_승하차_인원_정보(09월).csv', encoding='cp949')
df10 = pd.read_csv('/content/drive/MyDrive/project1_login/data/bus_station_boarding/2022년_버스노선별_정류장별_시간대별_승하차_인원_정보(10월).csv', encoding='cp949')
df11 = pd.read_csv('/content/drive/MyDrive/project1_login/data/bus_station_boarding/2022년_버스노선별_정류장별_시간대별_승하차_인원_정보(11월).csv', encoding='cp949')
df12 = pd.read_csv('/content/drive/MyDrive/project1_login/data/bus_station_boarding/2022년_버스노선별_정류장별_시간대별_승하차_인원_정보(12월).csv', encoding='cp949')

stop_loc = pd.read_excel('/content/drive/MyDrive/project1_login/data/bus_station_boarding/bus_stop_location_seoul.xlsx')

In [ ]:
df1.head()

In [ ]:
df1.info()

In [ ]:
# 컬렴명 비교
# 3,4,5,6월의 사용년월 컬럼이름이 다름 - 일치 시켜줄 것
# 8월부터 추가된 필요없는 추가 컬럼 제거 - '교통수단타입코드', '교통수단타입명', '등록일자'
print(df1.columns)
print(df2.columns)
print(df3.columns)
print(df4.columns)
print(df5.columns)
print(df6.columns)
print(df7.columns)
print(df8.columns)
print(df9.columns)
print(df10.columns)
print(df11.columns)

In [11]:
stop_loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11226 entries, 0 to 11225
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   NODE_ID  11226 non-null  int64  
 1   ARS-ID   11226 non-null  int64  
 2   정류소명     11226 non-null  object 
 3   X좌표      11226 non-null  float64
 4   Y좌표      11226 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 438.6+ KB


In [ ]:
df1.head(2)

In [ ]:
len(df1['표준버스정류장ID'].unique())

In [ ]:
df1['버스정류장ARS번호'].value_counts()

In [147]:
# 데이터 전처리 함수
def data_cleaning(df):
  # 필요없는 컬럼 드롭
  df.drop(labels=['노선명', '등록일자'], axis=1, inplace=True)
  
  if '교통수단타입코드' in df.columns:
    df.drop('교통수단타입코드', axis=1, inplace=True)
  
  if '교통수단타입명' in df.columns:
    df.drop('교통수단타입명', axis=1, inplace=True)
    
  # 컬럼명 통합
  if '?"사용년월"' in df.columns:
    df = df.rename(columns={'?"사용년월"':'사용년월', '표준버스정류장ID': 'node_id', '버스정류장ARS번호': 'ars_id'})
  else:
    df = df.rename(columns={'표준버스정류장ID': 'node_id', '버스정류장ARS번호': 'ars_id'})
  
  # 사용년월 결측값 채우기(ffill)
  df['사용년월'] = df['사용년월'].fillna(method='ffill')
  
  # 변수 타입 변경
  df['사용년월'] = df['사용년월'].astype('str')
  # ARS_ID의 값중 '~'가 포함된 정류장은 실제 승하차가 이뤄지지 않는 위치 추적용 가상 정류장이므로 드롭 후 int로 형변환
  drop_index = (df[df['ars_id'] == '~']).index
  df = df.drop(drop_index, axis=0)
  df['ars_id'] = df['ars_id'].astype('int')

  # 승하차 총합계 컬럼 추가
  df['승차총합계'] = df.iloc[:, 5::2].apply(np.sum, axis=1)
  df['하차총합계'] = df.iloc[:, 6::2].apply(np.sum, axis=1)

  return df

In [148]:
df1 = data_cleaning(df1)
df2 = data_cleaning(df2)
df3 = data_cleaning(df3)
df4 = data_cleaning(df4)
df5 = data_cleaning(df5)
df6 = data_cleaning(df6)
df7 = data_cleaning(df7)
df8 = data_cleaning(df8)
df9 = data_cleaning(df9)
df10 = data_cleaning(df10)
df11 = data_cleaning(df11)
df12 = data_cleaning(df12)

In [149]:
df7.columns

Index(['사용년월', '노선번호', 'node_id', 'ars_id', '역명', '00시승차총승객수', '00시하차총승객수',
       '1시승차총승객수', '1시하차총승객수', '2시승차총승객수', '2시하차총승객수', '3시승차총승객수', '3시하차총승객수',
       '4시승차총승객수', '4시하차총승객수', '5시승차총승객수', '5시하차총승객수', '6시승차총승객수', '6시하차총승객수',
       '7시승차총승객수', '7시하차총승객수', '8시승차총승객수', '8시하차총승객수', '9시승차총승객수', '9시하차총승객수',
       '10시승차총승객수', '10시하차총승객수', '11시승차총승객수', '11시하차총승객수', '12시승차총승객수',
       '12시하차총승객수', '13시승차총승객수', '13시하차총승객수', '14시승차총승객수', '14시하차총승객수',
       '15시승차총승객수', '15시하차총승객수', '16시승차총승객수', '16시하차총승객수', '17시승차총승객수',
       '17시하차총승객수', '18시승차총승객수', '18시하차총승객수', '19시승차총승객수', '19시하차총승객수',
       '20시승차총승객수', '20시하차총승객수', '21시승차총승객수', '21시하차총승객수', '22시승차총승객수',
       '22시하차총승객수', '23시승차총승객수', '23시하차총승객수', '승차총합계', '하차총합계'],
      dtype='object')

In [ ]:
df7.info()

## 월평균 유동인구총합 구하기

In [153]:
# df_list = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12]
# df = pd.merge(df_list, )ㄴㅇㄹㅁㄴㅁㄴㅇㄹㅁㄴㅇㄹㅁㄴㅇㄹㅁㄴㅇㄹㅁㄴㅇㄹㄴㅁㅇㄹㅁㄴㅇㄹㅁㄴㅇㄹㅁㄴㅇㄹㅇㄴㅁㄹㅁㄴㅇㄹㄴㅁㅇㄹㅁㄴㅇㄹㅁㄴㅇㄹㄴㅇㅁㄹㅁㄴㅇㄹㅁㄴㅇㄹㅁㄴㅇㄹ